In [ ]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import os
import sys
import platform
from copy import deepcopy
from typing import List, Dict, Optional, Union, Callable
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

# # required to enable non-blocking interaction:
# %gui qt5 ## TODO 2024-01-18 - this causes kernel to crash when running notebook remotely via VSCode's ssh remote

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.exception_helpers import CapturedException

# Jupyter interactivity:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget
from pyphocorehelpers.gui.Jupyter.simple_widgets import code_block_widget

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch

from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler, SavingOptions
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionTables, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import build_vscode_workspace, build_windows_powershell_run_script

from pyphocorehelpers.exception_helpers import CapturedException
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult, BatchSessionCompletionHandler

from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots, BatchPhoJonathanFiguresHelper
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsHelpers
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, filesystem_path_folder_contents_widget

import inspect
from jinja2 import Template
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import MAIN_get_template_string, write_test_script
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import export_session_h5_file_completion_function, curr_runtime_context_header_template, export_rank_order_results_completion_function, figures_rank_order_results_completion_function, determine_session_t_delta_completion_function, perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function, compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function, compute_and_export_session_wcorr_shuffles_completion_function, compute_and_export_session_instantaneous_spike_rates_completion_function, compute_and_export_session_extended_placefield_peak_information_completion_function, compute_and_export_session_alternative_replay_wcorr_shuffles_completion_function, backup_previous_session_files_completion_function, compute_and_export_session_trial_by_trial_performance_completion_function, save_custom_session_files_completion_function, compute_and_export_cell_first_spikes_characteristics_completion_function, figures_plot_cell_first_spikes_characteristics_completion_function
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import kdiba_session_post_fixup_completion_function
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import ProcessingScriptPhases

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _add_cell_remapping_category
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import BatchScriptsCollection, generate_batch_single_session_scripts, display_generated_scripts_ipywidget


BATCH_DAY_DATE: str = '2025-03-03'
# BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_Apogee' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}' # used for filenames throught the notebook
BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_GL' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_Lab' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_rMBP' # used for filenames throught the notebook

known_scripts_output_paths = [Path(v).resolve() for v in ['/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/cloud/turbo/Data/Output/gen_scripts', '/home/halechr/cloud/turbo/Data/Output/gen_scripts/', '/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', '/home/halechr/FastData/gen_scripts/', 'output/gen_scripts/', "K:/scratch/gen_scripts"]]
scripts_output_path = find_first_extant_path(known_scripts_output_paths)
assert scripts_output_path.exists(), f"scripts_output_path: {scripts_output_path} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'scripts_output_path: {scripts_output_path}')

collected_outputs_path = scripts_output_path.joinpath('../collected_outputs').resolve()
collected_outputs_path.mkdir(exist_ok=True)
assert collected_outputs_path.exists()
print(f'collected_outputs_path: {collected_outputs_path}')

## Build Processing Scripts:

In [2]:
# Get the generated code for user-contributed functions:
# phase_any_run_custom_user_completion_functions_dict = None
# phase_any_run_custom_user_completion_functions_dict = {} # do no post-user functions

phase_any_run_custom_user_completion_functions_dict = {
	# 'backup_previous_session_files_completion_function': backup_previous_session_files_completion_function,
    # "export_rank_order_results_completion_function": export_rank_order_results_completion_function, # ran 2024-09-26 3pm
# # # #     # "figures_rank_order_results_completion_function": figures_rank_order_results_completion_function,
    # "determine_session_t_delta_completion_function": determine_session_t_delta_completion_function,  # ran 2024-05-28 6am
	# 'perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function': perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function, # ran 2024-09-26 3pm
    # 'compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function': compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function, # ran 2024-09-26 3pm
    # 'kdiba_session_post_fixup_completion_function': kdiba_session_post_fixup_completion_function,
	# 'compute_and_export_session_wcorr_shuffles_completion_function': compute_and_export_session_wcorr_shuffles_completion_function,
	# 'compute_and_export_session_alternative_replay_wcorr_shuffles_completion_function': compute_and_export_session_alternative_replay_wcorr_shuffles_completion_function,
    # 'compute_and_export_session_instantaneous_spike_rates_completion_function': compute_and_export_session_instantaneous_spike_rates_completion_function,
    # 'compute_and_export_session_extended_placefield_peak_information_completion_function': compute_and_export_session_extended_placefield_peak_information_completion_function,
	# 'compute_and_export_session_trial_by_trial_performance_completion_function': compute_and_export_session_trial_by_trial_performance_completion_function, 
	# 'export_session_h5_file_completion_function': export_session_h5_file_completion_function, # ran 2024-09-26 3pm
	'save_custom_session_files_completion_function': save_custom_session_files_completion_function,
	# 'compute_and_export_cell_first_spikes_characteristics_completion_function': compute_and_export_cell_first_spikes_characteristics_completion_function,
	# 'figures_plot_cell_first_spikes_characteristics_completion_function': figures_plot_cell_first_spikes_characteristics_completion_function,
}

In [ ]:

# Hardcoded included_session_contexts:
included_session_contexts: List[IdentifyingContext] = UserAnnotationsManager.get_hardcoded_good_sessions()
# included_session_contexts
# excluded_session_keys = ['kdiba_gor01_one_2006-6-08_14-26-15', 'kdiba_gor01_two_2006-6-07_16-40-19', 'kdiba_pin01_one_fet11-01_12-58-54']
# excluded_session_keys = ['2006-6-08_14-26-15', '2006-6-09_1-22-43']
excluded_session_keys = []
excluded_session_contexts = [IdentifyingContext.try_init_from_session_key(session_str=v) for v in excluded_session_keys]
# excluded_session_contexts

included_session_context = [v for v in included_session_contexts if v not in excluded_session_contexts]
included_session_context

## Setup Functions:
# force_recompute_override_computations_includelist = ['rank_order_shuffle_analysis', '_add_extended_pf_peak_information',
#     '_build_trial_by_trial_activity_metrics',
#     '_decode_and_evaluate_epochs_using_directional_decoders',
#     '_decode_continuous_using_directional_decoders',
#     '_decoded_epochs_heuristic_scoring',
#     '_split_train_test_laps_data',
#     'perform_wcorr_shuffle_analysis'
# ]

force_recompute_override_computations_includelist = None
# force_recompute_override_computations_includelist = ['directional_decoders_evaluate_epochs', 'directional_decoders_epoch_heuristic_scoring']
# force_recompute_override_computations_includelist = ['split_to_directional_laps','merged_directional_placefields','directional_decoders_evaluate_epochs','directional_decoders_epoch_heuristic_scoring']
# force_recompute_override_computations_includelist = ['directional_decoders_evaluate_epochs','directional_decoders_epoch_heuristic_scoring']
# force_recompute_override_computations_includelist = ['lap_direction_determination', 'split_to_directional_laps', 'pf_computation', 'pfdt_computation','firing_rate_trends','merged_directional_placefields','rank_order_shuffle_analysis',]
# force_recompute_override_computations_includelist = ['firing_rate_trends', 'extended_stats', 'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', ]
# force_recompute_override_computations_includelist = ['rank_order_shuffle_analysis','wcorr_shuffle_analysis','trial_by_trial_metrics']

# force_recompute_override_computation_kwargs_dict = None
# force_recompute_override_computation_kwargs_dict = {'rank_order_shuffle_analysis':({'num_shuffles': 500, 'skip_laps': False} | dict(minimum_inclusion_fr_Hz=2.0, included_qclu_values=[1,2,4,5,6,7]))}
# computation_kwargs_dict={'rank_order_shuffle_analysis':({'num_shuffles': 500, 'skip_laps': False} | dict(minimum_inclusion_fr_Hz=2.0, included_qclu_values=[1,2,4,5,6,7]))}


extra_extended_computations_include_includelist = [
	# 'wcorr_shuffle_analysis'
]

custom_phase_extended_computations_include_includelist = None
# custom_phase_extended_computations_include_includelist=['lap_direction_determination', 'pf_computation', 'pfdt_computation', 'firing_rate_trends',
# 	'pf_dt_sequential_surprise',
# 	'extended_stats',
# 	'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 
# 	'ratemap_peaks_prominence2d',
# 	'long_short_inst_spike_rate_groups',
# 	'long_short_endcap_analysis',
# 	# 'spike_burst_detection',
# 	'split_to_directional_laps',
# 	'merged_directional_placefields',
# 	'rank_order_shuffle_analysis',
# 	# 'directional_train_test_split',
# 	'directional_decoders_decode_continuous',
# 	'directional_decoders_evaluate_epochs',
# 	'directional_decoders_epoch_heuristic_scoring',
#     'wcorr_shuffle_analysis',
#     'trial_by_trial_metrics',
#     'extended_pf_peak_information',
# ]


# debug_print = True
debug_print = False
known_global_data_root_parent_paths = [Path('/Users/pho/data'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'W:/Data'), Path(r'/home/halechr/FastData'), Path(r'/Volumes/MoverNew/data')] # Path(r'/home/halechr/cloud/turbo/Data'), , Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/home/halechr/turbo/Data'), 
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

## Different run configurations:
# ==================================================================================================================== #
# ACTIVE PHASE                                                                                                         #
# ==================================================================================================================== #

# active_phase = ProcessingScriptPhases.clean_run
# active_phase = ProcessingScriptPhases.continued_run
active_phase = ProcessingScriptPhases.final_run
# active_phase = ProcessingScriptPhases.figure_run


# END ________________________________________________________________________________________________________________ #
# custom_user_completion_functions_dict = active_phase.get_custom_user_completion_functions_dict(extra_run_functions=phase_any_run_custom_user_completion_functions_dict)
custom_user_completion_functions_dict = phase_any_run_custom_user_completion_functions_dict ## directly override the outputs:

# print(f'custom_user_completion_functions_dict: {custom_user_completion_functions_dict}')
custom_user_completion_function_template_code, custom_user_completion_functions_dict = MAIN_get_template_string(BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, collected_outputs_path=collected_outputs_path, override_custom_user_completion_functions_dict=custom_user_completion_functions_dict)
# print(f'custom_user_completion_function_template_code: {custom_user_completion_function_template_code}')
active_phase_dict = active_phase.get_run_configuration(custom_user_completion_function_template_code=custom_user_completion_function_template_code, extra_extended_computations_include_includelist=extra_extended_computations_include_includelist)

## Completely replace with custom functions:
if custom_phase_extended_computations_include_includelist is not None:
	print(f'WARNING: custom_phase_extended_computations_include_includelist: {custom_phase_extended_computations_include_includelist} is provided so only these extended_computation_fns will be used (overwritting the phase defaults!).')
	active_phase_dict['extended_computations_include_includelist'] = custom_phase_extended_computations_include_includelist

# active_phase_dict['extended_computations_include_includelist'].remove('wcorr_shuffle_analysis')
active_phase_dict['should_freeze_pipeline_updates'] = False 
# active_phase_dict['should_freeze_pipeline_updates'] = True # `should_freeze_pipeline_updates`
# active_phase_dict['should_create_vscode_workspace'] = True

# ## Default non-overriden for custom suffix:
# active_phase_dict['override_custom_pickle_suffix'] = '' # default, None
# active_phase_dict['force_recompute_override_computation_kwargs_dict'] = {}

# ==================================================================================================================== #
# Custom Suffix and overrides                                                                                          #
# ==================================================================================================================== #
# '_withNormalComputedReplays-frateThresh_5.0-qclu_[1, 2]' ___________________________________________________________ #
active_phase_dict['override_custom_pickle_suffix'] = '_withNormalComputedReplays-qclu_[1, 2]-frateThresh_5.0'
included_qclu_values = [1,2]
minimum_inclusion_fr_Hz = 5.0
job_suffix = f"_withNormalComputedReplays-qclu_12-frateThresh_5.0" ## ends up being something like # job_suffix = f"_withNormalComputedReplays-qclu_12-frateThresh_5.0_tbin_25ms"


# # # "_withNormalComputedReplays-frateThresh_5.0-qclu_[1, 2, 4, 6, 7, 9]" _______________________________________________ #
# active_phase_dict['override_custom_pickle_suffix'] = "_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0" ##
# included_qclu_values = [1, 2, 4, 6, 7, 9]
# minimum_inclusion_fr_Hz = 5.0
# job_suffix = f"_withNormalComputedReplays-qclu_124679-frateThresh_5.0"


# # "_withNormalComputedReplays-frateThresh_5.0-qclu_[1, 2]" _______________________________________________ #
# active_phase_dict['override_custom_pickle_suffix'] = "_withNormalComputedReplays-qclu_[1, 2]-frateThresh_5.0" 
# included_qclu_values = [1, 2]
# minimum_inclusion_fr_Hz = 5.0
# job_suffix = f"_withNormalComputedReplays-qclu_12-frateThresh_5.0"

# # # "_withNormalComputedReplays-frateThresh_5.0-qclu_[1, 2, 4, 6, 7, 8, 9]" _______________________________________________ #
# active_phase_dict['override_custom_pickle_suffix'] = "_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 8, 9]-frateThresh_5.0"
# included_qclu_values = [1, 2, 4, 6, 7, 8, 9]
# minimum_inclusion_fr_Hz = 5.0
# job_suffix = f"_withNormalComputedReplays-qclu_1246789-frateThresh_5.0"

# ==================================================================================================================== #
# Common:                                                                                                              #
# ==================================================================================================================== #
num_shuffles = 1024
drop_previous_result_and_compute_fresh = True

# laps_decoding_time_bin_size = None
laps_decoding_time_bin_size = 0.025
# laps_decoding_time_bin_size = 0.025

# ripple_decoding_time_bin_size = 0.058
# job_suffix='tbin_58ms'
ripple_decoding_time_bin_size = 0.025
# job_suffix='tbin_50ms'
# ripple_decoding_time_bin_size = 0.025
# job_suffix='tbin_25ms'


# job_suffix = f"{active_phase_dict['override_custom_pickle_suffix']}_tbin_25ms"
job_suffix = f"{job_suffix}_tbin_25ms"
# job_suffix = f"{job_suffix}_tbin_50ms"



# END ________________________________________________________________________________________________________________ #
print(f'job_suffix: {job_suffix}')
active_phase_dict['force_recompute_override_computation_kwargs_dict'] = {
	'merged_directional_placefields': {'laps_decoding_time_bin_size': laps_decoding_time_bin_size, 'ripple_decoding_time_bin_size': ripple_decoding_time_bin_size},
	'directional_decoders_evaluate_epochs': {'should_skip_radon_transform': False},
	'rank_order_shuffle_analysis': {'num_shuffles': num_shuffles, 'skip_laps': False, 'minimum_inclusion_fr_Hz':minimum_inclusion_fr_Hz, 'included_qclu_values':included_qclu_values},
    'perform_wcorr_shuffle_analysis': dict(num_shuffles=num_shuffles, drop_previous_result_and_compute_fresh=True),
}
# active_phase_dict['force_recompute_override_computations_includelist'] = list(active_phase_dict.get('force_recompute_override_computation_kwargs_dict', {}).keys())

# active_phase_dict['force_recompute_override_computations_includelist'] = [] # empty list so that everything isn't forced to recompute each time. NEEDS TO BE CHANGED WHEN qclus/fr_Hz change
# active_phase_dict['force_recompute_override_computations_includelist'] = ['directional_decoders_evaluate_epochs', 'directional_decoders_epoch_heuristic_scoring']
active_phase_dict['force_recompute_override_computations_includelist'] = list(active_phase_dict.get('force_recompute_override_computation_kwargs_dict', {}).keys())

active_phase_dict['override_parameters_flat_keypaths_dict'] = {'rank_order_shuffle_analysis.included_qclu_values': included_qclu_values, 'rank_order_shuffle_analysis.minimum_inclusion_fr_Hz': minimum_inclusion_fr_Hz,
} # need to provide `override_parameters_flat_keypaths_dict`

active_phase_dict['custom_user_completion_function_override_kwargs_dict'] = {
	'compute_and_export_session_alternative_replay_wcorr_shuffles_completion_function': dict(included_qclu_values=included_qclu_values, minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz, ripple_decoding_time_bin_size=ripple_decoding_time_bin_size, num_wcorr_shuffles=num_shuffles, drop_previous_result_and_compute_fresh=drop_previous_result_and_compute_fresh),
	# 'backup_previous_session_files_completion_function': dict(desired_suffix='Pre2024-11-21'),
	# 'compute_and_export_session_instantaneous_spike_rates_completion_function': dict(instantaneous_time_bin_size_seconds_list=[1000.0], epoch_handling_mode='UseAllEpochsMode', save_hdf=True, save_pickle=True, save_across_session_hdf=False),
	# 'compute_and_export_session_wcorr_shuffles_completion_function': dict(should_skip_previous_saved_shuffles=False, with_data_name=None),
	# 'perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function': dict(save_hdf=True, desired_shared_decoding_time_bin_sizes=None, custom_all_param_sweep_options=None, additional_session_context=None),
	# 'compute_and_export_session_trial_by_trial_performance_completion_function': dict(active_laps_decoding_time_bin_size=0.25),
    'compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function': dict(ripple_decoding_time_bin_size_override=ripple_decoding_time_bin_size, laps_decoding_time_bin_size_override=laps_decoding_time_bin_size,
        needs_recompute_heuristics=True, force_recompute_all_decoding=True,
        save_hdf=True, allow_append_to_session_h5_file=False,
		max_ignore_bins=2, same_thresh_cm=10.7, max_jump_distance_cm=60.0, # Heuristic settings
		),
}


# ==================================================================================================================== #
# Last-Chance Overrides                                                                                                #
# ==================================================================================================================== #
# # Interactive Debugging in Notebooks:
# active_phase_dict.update(dict(should_use_neptune_logging=False, should_create_vscode_workspace=True, should_use_file_redirected_output_logging=False, 
# 	should_freeze_pipeline_updates=True,
# ))

# ## No custom suffix!
# active_phase_dict.update(dict(job_suffix=job_suffix, should_freeze_pipeline_updates=False, should_use_neptune_logging=False, should_generate_run_notebooks=True, should_create_vscode_workspace=True, should_use_file_redirected_output_logging=False, create_slurm_scripts=True,
# ))


# Custom Job Suffix that will not interfere with running jobs ________________________________________________________ #
active_phase_dict.update(dict(job_suffix=job_suffix, should_freeze_pipeline_updates=True, should_use_neptune_logging=False, create_slurm_scripts=True, should_generate_run_scripts=True, should_generate_run_notebooks=True, should_create_vscode_workspace=True, should_use_file_redirected_output_logging=True,
))


# #TODO 2024-11-20 07:12: - [ ] Run with this, I hardcoded the parameter change for now
# {'ripple_decoding_time_bin_size': 0.058}

## Non-interactive batch computation in scripts
# active_phase_dict.update(dict(should_use_neptune_logging=True, should_generate_run_notebooks=False, should_use_file_redirected_output_logging=True))

# END ________________________________________________________________________________________________________________ #


# ==================================================================================================================== #
# BEGIN MAIN SCRIPT BODY                                                                                               #
# ==================================================================================================================== #
## Build Slurm Scripts:
session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
batch_scripts_collection: BatchScriptsCollection = generate_batch_single_session_scripts(global_data_root_parent_path, session_batch_basedirs=session_basedirs_dict, included_session_contexts=included_session_contexts,
																						output_directory=scripts_output_path, use_separate_run_directories=True,
																						# should_use_neptune_logging=False,
																						# should_use_neptune_logging=True,
																						# should_freeze_pipeline_updates=False, 
																						# create_slurm_scripts=True, 
																						# should_create_vscode_workspace=True,
																						# should_generate_run_notebooks=False,
																						# extended_computations_include_includelist=extended_computations_include_includelist,
																						# force_recompute_override_computations_includelist=force_recompute_override_computations_includelist, # ['split_to_directional_laps', 'rank_order_shuffle_analysis'],
																						# force_recompute_override_computation_kwargs_dict=force_recompute_override_computation_kwargs_dict,
																						# should_use_file_redirected_output_logging=False,
																						# should_use_file_redirected_output_logging=True,
																						# should_perform_figure_generation_to_file=False,
																						# batch_session_completion_handler_kwargs=dict(enable_hdf5_output=False),
																						# custom_user_completion_functions=custom_user_completion_functions,
																						# custom_user_completion_function_template_code=custom_user_completion_function_template_code,
																						# should_force_reload_all=True,
																						# should_force_reload_all=False,
																						**active_phase_dict
																					)


# batch_scripts_collection.included_session_contexts, output_python_scripts, output_slurm_scripts

output_python_scripts = batch_scripts_collection.output_python_scripts
output_jupyter_notebooks = batch_scripts_collection.output_jupyter_notebooks
output_slurm_scripts = batch_scripts_collection.output_slurm_scripts
vscode_workspace_path = batch_scripts_collection.vscode_workspace_path

if vscode_workspace_path is not None:
	display(fullwidth_path_widget(vscode_workspace_path, file_name_label='vscode_workspace_path:'))

print(f"extended_computations_include_includelist: {active_phase_dict['extended_computations_include_includelist']}")
computation_script_paths = [x[0] for x in output_python_scripts]
generate_figures_script_paths = [x[1] for x in output_python_scripts]
print(f'generate_figures_script_paths: {generate_figures_script_paths}')
_out_widget = display_generated_scripts_ipywidget(batch_scripts_collection.included_session_contexts, output_python_scripts)
display(_out_widget)

In [ ]:
## INPUTS: active_phase, output_slurm_scripts, computation_script_paths, generate_figures_script_paths
also_show_figure_script_outputs: bool = False

# Maximum number of parallel script executions
max_parallel_executions = 1
# List of your script paths
# if active_phase.value == ProcessingScriptPhases.figure_run:
if active_phase.is_figure_phase:
	print(f'fig mode!')
	script_paths = generate_figures_script_paths
else:
	script_paths = computation_script_paths


if (platform.system() == 'Windows'):
	powershell_script_path = build_windows_powershell_run_script(script_paths, max_concurrent_jobs=max_parallel_executions, script_name='run_scripts')
	# print(f'powershell_script_path: {powershell_script_path}')
	display(fullwidth_path_widget(powershell_script_path, file_name_label='powershell_script_path:'))

	if also_show_figure_script_outputs or active_phase_dict['should_perform_figure_generation_to_file']:
		powershell_figures_script_path = build_windows_powershell_run_script(generate_figures_script_paths, max_concurrent_jobs=1, script_name='run_figure_scripts')
		display(fullwidth_path_widget(powershell_figures_script_path, file_name_label='powershell_figures_script_path:'))


if (platform.system() != 'Windows'):
	## Linux Only: Slurm Scripts
	# sbatch_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts])
	sbatch_run_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts['run']])
	sbatch_figs_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts['figs']])

	if not active_phase.is_figure_phase:
		# Create and display the code block widget
		run_slurm_code_block = code_block_widget(sbatch_run_start_slurm_scripts, label="RunMain:")

	if (active_phase.is_figure_phase or also_show_figure_script_outputs and (len(sbatch_figs_start_slurm_scripts)>0)):
		# Create and display the code block widget
		figs_slurm_code_block = code_block_widget(sbatch_figs_start_slurm_scripts, label="Figures:")

# Batch 1
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-02_17-46-44/run_kdiba_pin01_one_11-02_17-46-44.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-02_19-28-0/run_kdiba_pin01_one_11-02_19-28-0.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-03_12-3-25/run_kdiba_pin01_one_11-03_12-3-25.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_fet11-01_12-58-54/run_kdiba_pin01_one_fet11-01_12-58-54.sh'

# Batch 2:
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30/run_kdiba_vvp01_one_2006-4-09_17-29-30.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50/run_kdiba_vvp01_one_2006-4-10_12-25-50.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54/run_kdiba_vvp01_two_2006-4-09_16-40-54.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-10_12-58-3/run_kdiba_vvp01_two_2006-4-10_12-58-3.sh'

# Batch 3
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15/run_kdiba_gor01_one_2006-6-08_14-26-15.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/run_kdiba_gor01_one_2006-6-09_1-22-43.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19/run_kdiba_gor01_two_2006-6-07_16-40-19.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-08_21-16-25/run_kdiba_gor01_two_2006-6-08_21-16-25.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-09_22-24-40/run_kdiba_gor01_two_2006-6-09_22-24-40.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46/run_kdiba_gor01_two_2006-6-12_16-53-46.sh'

# KDIBA/gor01/one/2006-6-09_1-22-43
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/run_kdiba_gor01_one_2006-6-09_1-22-43.sh'

# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/run_kdiba_gor01_one_2006-6-09_1-22-43.sh'
# 
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15/run_kdiba_gor01_one_2006-6-08_14-26-15.sh'
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-12_15-55-31/run_kdiba_gor01_one_2006-6-12_15-55-31.sh'
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19/run_kdiba_gor01_two_2006-6-07_16-40-19.sh'
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46/run_kdiba_gor01_two_2006-6-12_16-53-46.sh'
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30/run_kdiba_vvp01_one_2006-4-09_17-29-30.sh'
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50/run_kdiba_vvp01_one_2006-4-10_12-25-50.sh'
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54/run_kdiba_vvp01_two_2006-4-09_16-40-54.sh'
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-10_12-58-3/run_kdiba_vvp01_two_2006-4-10_12-58-3.sh'
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_pin01_one_11-03_12-3-25/run_kdiba_pin01_one_11-03_12-3-25.sh'
# sbatch '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_pin01_one_fet11-01_12-58-54/run_kdiba_pin01_one_fet11-01_12-58-54.sh'


In [ ]:
import jinja2
print(jinja2.__version__)


## Execute the generated scripts in parallel:

In [ ]:
import subprocess
from concurrent.futures import ProcessPoolExecutor, as_completed
import ipywidgets as widgets
from IPython.display import display

## recieves: max_parallel_executions, script_paths
"""
# Maximum number of parallel script executions
max_parallel_executions = 5
# List of your script paths
script_paths = output_python_scripts
"""

# Function to execute a script
def run_script(script_path):
	python_executable = 'python'
	# python_executable = '/home/halechr/Library/VSCode/green/.venv_green/bin/python'
	
	try:
		result = subprocess.run([python_executable, script_path], capture_output=True, text=True)
		return script_path, result.stdout, result.stderr
	except BaseException as e:
		return script_path, None, str(e)

# Create a progress bar
progress_bar = widgets.IntProgress(value=0, min=0, max=len(script_paths), description='Running:', bar_style='info')
display(progress_bar)

# Run scripts in parallel with a limit on the number of parallel instances
with ProcessPoolExecutor(max_workers=max_parallel_executions) as executor:
	futures = {executor.submit(run_script, path): path for path in script_paths}
	for future in as_completed(futures):
		script, stdout, stderr = future.result()
		progress_bar.value += 1  # Update the progress bar
		if stderr:
			print(f"Error in {script}: {stderr}")
		else:
			print(f"Completed {script}")

# Progress bar will automatically update as scripts complete



# 🔷 Maintenence and File Management

In [ ]:
import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from neuropy.utils.result_context import IdentifyingContext
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files, OldFileArchiver
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionHelpers

all_known_session_contexts: List[IdentifyingContext] = UserAnnotationsManager.get_all_known_sessions()
known_global_data_root_parent_paths = [Path('/Users/pho/data'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'W:/Data'), Path(r'/home/halechr/FastData'), Path(r'/Volumes/MoverNew/data')] # Path(r'/home/halechr/cloud/turbo/Data'), , Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/home/halechr/turbo/Data'), 
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
print(f'global_data_root_parent_path: {global_data_root_parent_path}')
all_known_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, all_known_session_contexts)
known_collected_outputs_paths = [Path(v).resolve() for v in ['/Users/pho/data/collected_outputs',
                                                            '/Volumes/SwapSSD/Data/collected_outputs', r"K:/scratch/collected_outputs", '/Users/pho/Dropbox (University of Michigan)/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs',
                                                            '/home/halechr/FastData/collected_outputs/', '/home/halechr/cloud/turbo/Data/Output/collected_outputs']]
collected_outputs_directory = find_first_extant_path(known_collected_outputs_paths)
assert collected_outputs_directory.exists(), f"collected_outputs_directory: {collected_outputs_directory} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'collected_outputs_directory: {collected_outputs_directory}')


known_scripts_output_paths = [Path(v).resolve() for v in ['/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/cloud/turbo/Data/Output/gen_scripts', '/home/halechr/cloud/turbo/Data/Output/gen_scripts/', '/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', '/home/halechr/FastData/gen_scripts/', 'output/gen_scripts/', "K:/scratch/gen_scripts"]]
scripts_output_path = find_first_extant_path(known_scripts_output_paths)
assert scripts_output_path.exists(), f"scripts_output_path: {scripts_output_path} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'scripts_output_path: {scripts_output_path}')


## Move misplaced files to session folders

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import try_move_pickle_files_on_GL
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionHelpers

## INPUTS: good_session_concrete_folders, session_basedirs_dict, computation_script_paths

# session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
is_dryrun: bool = True
# is_dryrun: bool = False
debug_print: bool = False

# copy_dict, moved_dict, (all_found_pipeline_pkl_files_dict, all_found_global_pkl_files_dict, all_found_pipeline_h5_files_dict) = try_move_pickle_files_on_GL(good_session_concrete_folders, session_basedirs_dict, computation_script_paths,
# 			is_dryrun=is_dryrun, debug_print=debug_print)

## Specify which files to match:    
copy_dict, moved_files_dict_files = AcrossSessionHelpers._copy_exported_files_from_session_folder_to_collected_outputs(BATCH_DATE_TO_USE='2025-03-04', cuttoff_date=datetime(2025, 2, 20), target_dir=collected_outputs_directory, custom_file_globs_dict={
    'pkl': '*standalone_wcorr_ripple_shuffle_data_only*.pkl',
    'csv': '*ripple_WCorrShuffle_df*.csv',
    # 'h5': '*.h5',
}, is_dry_run=is_dryrun)

copy_dict

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionHelpers

is_dryrun: bool = False

assert collected_outputs_directory.exists()
copy_dict, moved_files_dict_files = AcrossSessionHelpers._copy_exported_files_from_session_folder_to_collected_outputs(BATCH_DATE_TO_USE='2025-03-04', cuttoff_date=datetime(2024, 11, 25), target_dir=collected_outputs_directory, is_dry_run=is_dryrun)
copy_dict


In [ ]:


script_output_folders = [Path(v).parent for v in computation_script_paths]

excluded_session_keys = ['kdiba_pin01_one_fet11-01_12-58-54', 'kdiba_gor01_one_2006-6-08_14-26-15', 'kdiba_gor01_two_2006-6-07_16-40-19']
excluded_session_contexts = [IdentifyingContext(**dict(zip(IdentifyingContext._get_session_context_keys(), v.split('_', maxsplit=3)))) for v in excluded_session_keys]
# excluded_session_contexts

all_found_pkl_files_dict = {}
all_found_pipeline_pkl_files_dict = {}
all_found_global_pkl_files_dict = {}
all_found_pipeline_h5_files_dict = {}

copy_dict = {}
moved_dict = {}

# scripts_output_path
for a_good_session_concrete_folder, a_session_basedir, a_script_folder in zip(good_session_concrete_folders, session_basedirs_dict, script_output_folders):
	if debug_print:
		print(f'a_good_session_concrete_folder: {a_good_session_concrete_folder}, a_session_basedir: {a_session_basedir}. a_script_folder: {a_script_folder}')
	if a_good_session_concrete_folder.context in excluded_session_contexts:
		if debug_print:
			print(f'skipping excluded session: {a_good_session_concrete_folder.context}')
	else:
		all_found_global_pkl_files_dict[a_session_basedir] = list(a_script_folder.glob('global_computation_results*.pkl'))
		
		for a_global_file in all_found_global_pkl_files_dict[a_session_basedir]:
			## iterate through the found global files:
			target_file = a_good_session_concrete_folder.global_computation_result_pickle.with_name(a_global_file.name)
			copy_dict[a_global_file] = target_file
			# if not is_dryrun:
			## perform the move/copy
			was_success = try_perform_move(src_file=a_global_file, target_file=target_file, is_dryrun=is_dryrun)
			if was_success:
				moved_dict[a_file] = target_file
		all_found_pipeline_pkl_files_dict[a_session_basedir] = list(a_script_folder.glob('loadedSessPickle*.pkl'))
		for a_file in all_found_pipeline_pkl_files_dict[a_session_basedir]:
			## iterate through the found global files:
			target_file = a_good_session_concrete_folder.session_pickle.with_name(a_file.name)
			copy_dict[a_file] = target_file
			# if not is_dryrun:
			## perform the move/copy
			was_success = try_perform_move(src_file=a_file, target_file=target_file, is_dryrun=is_dryrun)
			if was_success:
				moved_dict[a_file] = target_file
		all_found_pipeline_h5_files_dict[a_session_basedir] = list(a_script_folder.glob('loadedSessPickle*.h5'))
		for a_file in all_found_pipeline_h5_files_dict[a_session_basedir]:
			## iterate through the found global files:
			target_file = a_good_session_concrete_folder.pipeline_results_h5.with_name(a_file.name)
			copy_dict[a_file] = target_file
			# if not is_dryrun:
			## perform the move/copy
			was_success = try_perform_move(src_file=a_file, target_file=target_file, is_dryrun=is_dryrun)
			if was_success:
				moved_dict[a_file] = target_file
		# all_found_pkl_files_dict[a_session_basedir] = find_pkl_files(a_script_folder)

## discover .pkl files in the root of each folder:
# all_found_pipeline_pkl_files_dict
# all_found_global_pkl_files_dict
## OUTPUTS: copy_dict
# copy_dict
moved_dict

## ⛓️🟢 Post-Processing of batch outputs:

In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import save_filelist_to_text_file


debug_print = True
known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
# Hardcoded included_session_contexts:
included_session_contexts = UserAnnotationsManager.get_hardcoded_good_sessions()
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

# Output Paths:
## OUTPUTS: included_h5_paths, included_session_contexts, good_session_concrete_folders

## INPUTS: good_session_concrete_folders, target_dir, BATCH_DATE_TO_USE

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path

included_h5_paths = [Path(get_file_str_if_file_exists(v.pipeline_results_h5)).resolve() for v in good_session_concrete_folders]
check_output_h5_files(included_h5_paths)
included_h5_paths

def _across_session_h5_output_basename_fn(session_context: Optional[IdentifyingContext], session_descr: Optional[str], basename: str, *args, separator_char: str = "_"):
	""" Captures `BATCH_DATE_TO_USE` """
	# a_session_folder.context
	if session_context is not None:
		session_descr = session_context.session_name # '2006-6-07_16-40-19'
	_filename_list = [BATCH_DATE_TO_USE, session_descr, basename]
	if len(args) > 0:
		_filename_list.extend([str(a_part) for a_part in args if a_part is not None])
	return separator_char.join(_filename_list)

copy_h5_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, target_dir=collected_outputs_path, backup_mode=BackupMethods.CommonTargetDirectory, rename_backup_basename_fn=_across_session_h5_output_basename_fn, only_include_file_types=['h5']) # , rename_backup_suffix=BATCH_DATE_TO_USE
copy_h5_dict


## INPUTS: target_dir, BATCH_DATE_TO_USE
h5_filelist_output_filename=f'{BATCH_DATE_TO_USE}_all_sessions_h5_filelist.txt'
h5_filelist_output_file_path = Path(target_dir).joinpath(h5_filelist_output_filename).resolve() # Use Default
print(f'h5_filelist_output_file_path: {h5_filelist_output_file_path}')
_out_string, filelist_path = save_filelist_to_text_file(included_h5_paths, filelist_path=h5_filelist_output_file_path, debug_print=True) # r"W:\Data\all_sessions_h5_filelist_2024-03-28_Apogee.txt"
filelist_path

# Output File Processing Helpers

## Copy exported `pipeline_results.h5` files into the `collected_outputs` folder, adding the current date
```
copying "W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\pipeline_results.h5"
		 -> "K:\scratch\collected_outputs\2024-06-12_Apogee_2006-6-08_14-26-15_pipeline_results.h5"...
```

In [ ]:
a_copy_dict = copy_h5_dict

# Copies each file sequentially to the collected_outputs directory, and then builds an output file list
## INPUT a_copy_dict
moved_files_dict_h5_files = copy_movedict(a_copy_dict)
moved_files_dict_h5_files
# INPUTS: active_filelist_prefix, target_dir
# active_filelist_prefix: str = 'backed_up_files'
active_filelist_prefix: str = 'session_h5_files'

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path

moved_files_copydict_output_filename=f'{active_filelist_prefix}_copydict_{BATCH_DATE_TO_USE}.csv'
moved_files_copydict_file_path = target_dir.joinpath(moved_files_copydict_output_filename).resolve() # Use Default
print(f'moved_files_copydict_file_path: "{moved_files_copydict_file_path}"')

_out_string, filedict_out_path = save_copydict_to_text_file(moved_files_dict_h5_files, moved_files_copydict_file_path, debug_print=True)

## 2024-09-04 - Batch Output Files

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import copy_session_inst_fr_data_to_across_session_pkl

## Complete Concise Run:
# RESULT_DATE_TO_USE = '2024-09-26'

RESULT_DATE_TO_USE: str = BATCH_DAY_DATE
print(f'RESULT_DATE_TO_USE: {RESULT_DATE_TO_USE}')

# instantaneous_time_bin_size_seconds_list = [0.002, 0.005, 0.025, 1000.0]
instantaneous_time_bin_size_seconds_list = [1000.0]

across_sessions_recomputed_instantaneous_fr_dict, moved_files_dict_files, (filelist_path, filedict_out_path) = copy_session_inst_fr_data_to_across_session_pkl(RESULT_DATE_TO_USE=RESULT_DATE_TO_USE, collected_outputs_path=collected_outputs_path, instantaneous_time_bin_size_seconds_list=instantaneous_time_bin_size_seconds_list)


# '/nfs/turbo/umms-kdiba/Data/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-10_GL.pkl'
# Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

# ! After Sessions are processed and their `.h5` files are exported, build the combined tables
The combined (across session) tables are saved out into .csv and .pkl formats: `W:\Data\2024-06-12_Apogee\neuron_identities_table.csv` -- ideally this would be in the combined outputs folder

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

## FILE_OUTPUTS: '/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22_GL'

## INPUTS: included_session_contexts, included_h5_paths, global_data_root_parent_path, BATCH_DATE_TO_USE
# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True, )

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path
assert target_dir.exists(), f'target_dir: "{target_dir}" does not exist!'
(neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths, override_output_parent_path=target_dir, output_path_suffix=f'{BATCH_DATE_TO_USE}')
curr_collected_outputs_folder = Path(output_path_dicts['neuron_replay_stats_table']['.csv']).resolve().parent
filesystem_path_folder_contents_widget(curr_collected_outputs_folder)

In [ ]:
neuron_replay_stats_table

# 2024-06-12 11:18: - [ ] HACK: reads back in the exported `neuron_replay_stats_table` CSV to fix an issue with pd.NA and np.nan not reading a NaNs and the dtypes of the columns being messed up on load:

In [ ]:
_curr_read_path = output_path_dicts['neuron_replay_stats_table']['.csv']
# _curr_read_path = Path('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/2024-06-12_Apogee/neuron_replay_stats_table.csv').resolve()
neuron_replay_stats_table: pd.DataFrame = pd.read_csv(_curr_read_path, na_values=['', 'nan', 'np.nan','<NA>'],
										dtype={
											'long_pf_peak_x': 'float64', 
											'short_pf_peak_x': 'float64',
											'long_LR_pf2D_peak_x': 'float64',
											'long_LR_pf2D_peak_y': 'float64',
											'long_RL_pf2D_peak_x': 'float64',
											'long_RL_pf2D_peak_y': 'float64',
											'short_LR_pf2D_peak_x': 'float64',
											'short_LR_pf2D_peak_y': 'float64',
											'short_RL_pf2D_peak_x': 'float64',
											'short_RL_pf2D_peak_y': 'float64',
											'long_LR_pf1D_peak': 'float64',
											'long_RL_pf1D_peak': 'float64',
											'short_LR_pf1D_peak': 'float64',
											'short_RL_pf1D_peak': 'float64',
											'peak_diff_RL_pf1D_peak': 'float64',
											'peak_diff_LR_pf1D_peak': 'float64'
										}, index_col='neuron_uid')
neuron_replay_stats_table


In [ ]:
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import TruncationCheckingResults

# truncation_checking_aclus_dict, jonathan_firing_rate_analysis_result.neuron_replay_stats_df = truncation_checking_result.build_truncation_checking_aclus_dict(neuron_replay_stats_df=jonathan_firing_rate_analysis_result.neuron_replay_stats_df)

# 'is_rate_extrema', 'is_refined_exclusive', 'is_refined_LxC', 'is_refined_SxC'
# 'is_long_peak_left_cap', 'is_long_peak_right_cap', 'is_long_peak_either_cap'

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _add_cell_remapping_category

neuron_replay_stats_df = deepcopy(neuron_replay_stats_table)

neuron_replay_stats_df, (non_disappearing_endcap_cells_df, disappearing_endcap_cells_df, minorly_changed_endcap_cells_df, significant_distant_remapping_endcap_cells_df) = _add_cell_remapping_category(neuron_replay_stats_df=neuron_replay_stats_df,
													   loaded_track_limits = {'long_xlim': np.array([59.0774, 228.69]), 'short_xlim': np.array([94.0156, 193.757]), 'long_ylim': np.array([138.164, 146.12]), 'short_ylim': np.array([138.021, 146.263])},
)
# neuron_replay_stats_df
significant_distant_remapping_endcap_cells_df

In [ ]:
neuron_replay_stats_df

In [ ]:
# non_disappearing_endcap_cells_df
significant_distant_remapping_endcap_cells_df
# 'track_membership'

## Required for Interactive Plotting and figure displace

In [ ]:
neuron_replay_stats_table.index.unique() #['neuron_uid'].unique()

In [ ]:

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
num_sessions: int = 13
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)

## Extract output files from all completed sessions:

# 2023-10-04 - Load Saved across-sessions-data, process, and produce figures

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

# 'collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl'
across_session_result_long_short_recomputed_inst_firing_rate_file = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
across_session_result_long_short_recomputed_inst_firing_rate_file.exists()

'across_session_result_long_short_inst_firing_rate_2024-06-11_GL.pkl'

In [ ]:
# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

## INPUTS: BATCH_DATE_TO_USE, BATCH_DATE_TO_USE
print(f"BATCH_DATE_TO_USE: {BATCH_DATE_TO_USE}")
## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
#                                                  inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')

across_session_result_long_short_recomputed_inst_firing_rate_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# "/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-10-22_GL.pkl"

# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_recomputed_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
#                                                  inst_fr_output_filename=across_session_result_long_short_recomputed_inst_firing_rate_filename)

AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_recomputed_instantaneous_fr_dict, global_data_root_parent_path=collected_outputs_path.resolve(),
												 inst_fr_output_filename=across_session_result_long_short_recomputed_inst_firing_rate_filename)


# "/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-09-26_GL.pkl"

# '/nfs/turbo/umms-kdiba/Data/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-10_GL.pkl'
# Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
# output_path_suffix: str = '2024-09-11_GL'
# output_path_suffix: str = '2024-09-26_GL'
output_path_suffix: str = BATCH_DATE_TO_USE
# graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=global_data_root_parent_path, BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, plotting_enabled=True)
graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=collected_outputs_path, output_path_suffix=output_path_suffix, plotting_enabled=True)


In [ ]:
from PendingNotebookCode import build_and_merge_all_sessions_joined_neruon_fri_df

all_sessions_joined_neruon_fri_df, out_path = build_and_merge_all_sessions_joined_neruon_fri_df(global_data_root_parent_path, BATCH_DATE_TO_USE, included_session_contexts=included_session_contexts)

In [ ]:
from neuropy.core.user_annotations import UserAnnotationsManager, SessionCellExclusivityRecord
from neuropy.utils.result_context import IdentifyingContext

annotation_man = UserAnnotationsManager()

LxC_uids = []
SxC_uids = []

for a_ctxt in included_session_contexts:
	session_uid = a_ctxt.get_description(separator="|", include_property_names=False)
	session_uid
	session_cell_exclusivity: SessionCellExclusivityRecord = annotation_man.annotations[a_ctxt].get('session_cell_exclusivity', None)
	LxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.LxC])
	SxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.SxC])
	
# [a_ctxt.get_description(separator="|", include_property_names=False) for a_ctxt in included_session_contexts]

long_short_fr_indicies_analysis_table['XxC_status'] = 'Shared'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, LxC_uids), 'XxC_status'] = 'LxC'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, SxC_uids), 'XxC_status'] = 'SxC'

long_short_fr_indicies_analysis_table
## 2023-10-11 - Get the long peak location

long_short_fr_indicies_analysis_table['long_pf_peak_x'] = neuron_replay_stats_table['long_pf_peak_x']
long_short_fr_indicies_analysis_table

## SAVE:
# long_short_fr_indicies_analysis_table_filename = 'output/2023-10-07_long_short_fr_indicies_analysis_table.csv'
long_short_fr_indicies_analysis_table_filename: str = f'output/{BATCH_DATE_TO_USE}_long_short_fr_indicies_analysis_table.csv'
long_short_fr_indicies_analysis_table.to_csv(long_short_fr_indicies_analysis_table_filename)
print(f'saved: {long_short_fr_indicies_analysis_table_filename}')

# Loading

In [ ]:
## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-21.pkl'
# inst_fr_output_filename=f'across_session_result_handler_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-08-09_Test.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-04-GL-Recomp.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-07.pkl'
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_load_filepath: Path = Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# inst_fr_output_load_filepath: Path = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
inst_fr_output_load_filepath: Path = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-09-26_GL.pkl').resolve()

# "/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-09-26_GL.pkl"
assert inst_fr_output_load_filepath.exists()
inst_fr_output_filename: str = inst_fr_output_load_filepath.name
# across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=inst_fr_output_load_filepath.parent, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
 
## Load all across-session tables from the pickles:
# output_path_suffix: str = f'2023-10-07'
# output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
# output_path_suffix: str = '2024-06-12_GL'
# output_path_suffix: str = '2024-09-03_GL'
output_path_suffix: str = '2024-09-26_GL'
# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=collected_outputs_path, output_path_suffix=output_path_suffix)
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')

### 2023-11-15 - For manual testing

In [ ]:
# ## Load the saved across-session results:
# Outputs: across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list, neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table

inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')
# neuron_replay_stats_table


In [ ]:
across_sessions_instantaneous_frs_list

# 2024-06-11 - Across Session Outputs

In [ ]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
long_short_fr_indicies_analysis_table.plot.scatter(x='long_pf_peak_x', y='x_frs_index', title='Pf Peak position vs. LapsFRI', ylabel='Lap FRI')
long_short_fr_indicies_analysis_table.plot.scatter(x='long_pf_peak_x', y='y_frs_index', title='Pf Peak position vs. ReplayFRI', ylabel='Replay FRI')

# 2023-10-10 - Statistics for `across_sessions_bar_graphs`, analysing `across_session_inst_fr_computation` 

In [ ]:
import scipy.stats as stats
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import pho_stats_perform_diagonal_line_binomial_test, pho_stats_bar_graph_t_tests

binom_test_chance_result = pho_stats_perform_diagonal_line_binomial_test(long_short_fr_indicies_analysis_table)
print(f'binom_test_chance_result: {binom_test_chance_result}')

LxC_Laps_T_result, SxC_Laps_T_result, LxC_Replay_T_result, SxC_Replay_T_result = pho_stats_bar_graph_t_tests(across_session_inst_fr_computation)

# n_total: 823, n_above_diagonal: 457, n_exact_on_diagonal: 0, n_below_diagonal: 366
# binom_test_chance_result: BinomTestResult(k=457, n=823, alternative='two-sided', statistic=0.5552855407047388, pvalue=0.0016893424059938723)
# LxC_Laps_T_result: TtestResult(statistic=12.249237714915296, pvalue=3.8446809431691085e-08, df=12)
# SxC_Laps_T_result: TtestResult(statistic=-12.413163641851535, pvalue=5.768107640498047e-07, df=9)
# LxC_Replay_T_result: TtestResult(statistic=-0.7636543941504783, pvalue=0.4598247399068105, df=12)
# SxC_Replay_T_result: TtestResult(statistic=-3.069767144140489, pvalue=0.01335902191105584, df=9)

# n_total: 823, n_above_diagonal: 457, n_exact_on_diagonal: 0, n_below_diagonal: 366
# binom_test_chance_result: BinomTestResult(k=457, n=823, alternative='two-sided', statistic=0.5552855407047388, pvalue=0.0016893424059938723)
# LxC_Laps_T_result: TtestResult(statistic=12.249237714915296, pvalue=3.8446809431691085e-08, df=12)
# SxC_Laps_T_result: TtestResult(statistic=-12.413163641851535, pvalue=5.768107640498047e-07, df=9)
# LxC_Replay_T_result: TtestResult(statistic=-0.7636543941504783, pvalue=0.4598247399068105, df=12)
# SxC_Replay_T_result: TtestResult(statistic=-3.069767144140489, pvalue=0.01335902191105584, df=9)


In [ ]:
debug_print: bool = False
enable_neptune: bool = True

if enable_neptune:
	import neptune # for logging progress and results
	from neptune.types import File
	from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables 

	## Gets the notebook filepath for Neptune:
	import IPython
	from pyphocorehelpers.programming_helpers import IPythonHelpers
	notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
	assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"

	
	# notebook_filepath

	# '/home/halechr/repos/Spike3D/BatchInteractiveProcessing_2024-06-11_GL.ipynb'

	neptune_kwargs = {'project':"commander.pho/PhoDibaLongShortAcrossSessions",
	'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
			
	neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])


	if neptuner.run is None:
		# neptuner.run = neptune.init_run(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath]) # see git_ref=GitRef(repository_path="/path/to/repo")
		# Add the session_context properties to the run: {'format_name': 'kdiba', 'animal': 'vvp01', 'exper_name': 'two', 'session_name': '2006-4-09_16-40-54'}

		neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])

		# Create an AutoValueConvertingNeptuneRun and copy the attributes
		# neptuner.run = AutoValueConvertingNeptuneRun(base_run._client, base_run._uuid, base_run._url)
	
		params = {'BATCH_DATE_TO_USE': BATCH_DATE_TO_USE, "run_workstation": "GL"}
		neptuner.run["parameters"] = params

		neptuner.outputs = neptuner.run['outputs']
		neptuner.figures = neptuner.outputs['figures']

	neptuner_run = neptuner.run
	
	# run = neptune.init_run(source_files=["**/*.dvc"])

	# # Pre-execution dataframe view:
	# run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"

else:
	# no neptune:
	neptuner = None    
	neptuner_run = None

In [ ]:
for a_save_path, a_save_dict in registered_output_files.items():
	a_save_dict['fig']
	a_save_dict['context']
	neptuner.figures.upload(File(a_save_path.resolve().as_posix()))

In [ ]:

neptuner_run[f'output_files/across_sessions_fig_2'] = _out_fig_2

In [ ]:
if (neptuner is not None) and (neptuner_run is not None):
	neptuner.stop()

# KDiba Session Discovery - Determines contexts to process - 2024-09-19

In [ ]:
# from pathlib import Path
# from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
# from neuropy.utils.result_context import print_identifying_context_array_code

# known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
# global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
# assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
sessions_df, export_folder_path = KDibaOldDataSessionFormatRegisteredClass.find_build_and_save_sessions_experiment_datetime_df_csv(
																			# global_data_root_parent_path=global_data_root_parent_path,
																			export_csv_path=Path('EXTERNAL/PhoDibaPaper2024Book/data/sessions_experiment_datetime_df.csv').resolve(),
																			bad_sessions_csv_path=Path('EXTERNAL/PhoDibaPaper2024Book/data/2024-09-23_bad_sessions_table.csv').resolve(),
)
sessions_df = sessions_df.reset_index(drop=True, inplace=False)
sessions_df


In [ ]:
# old_included_session_contexts = [
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), #
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
#     # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), #
# ]
	
# new_included_session_contexts = [
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_19-11-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_21-2-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_12-48-38'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-11_15-16-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_16-2-46'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_14-39-31'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_14-59-23'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_17-53-55'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_14-49-24'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-16_15-12-23'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-17_12-33-47'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_13-6-1'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_13-28-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_15-23-32'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_15-38-2'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_13-50-7'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_16-37-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_16-48-9'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-21_10-24-35'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-21_11-19-2'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_14-28-51'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_17-17-6'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_17-33-28'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-26_13-22-13'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-26_13-51-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-27_14-43-12'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-27_18-21-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_12-17-27'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_16-48-29'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_17-6-14'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-07_11-26-53'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_15-46-47'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_3-23-37'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_11-0-53'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_20-28-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_21-26-8'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-04_21-20-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-05_19-26-43'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_11-43-50'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_21-17-16'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_12-35-59'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-2-0'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-55-7'),
# ]

# completely_new_included_session_contexts = [
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_14-28-51'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_16-48-29'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_13-28-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_19-11-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-26_13-22-13'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_3-23-37'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_17-33-28'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_17-6-14'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_11-0-53'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_13-6-1'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_11-43-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-27_18-21-57'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_21-17-16'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-27_14-43-12'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_13-50-7'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-21_10-24-35'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_17-53-55'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_16-37-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-11_15-16-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_16-48-9'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-2-0'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-55-7'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_14-59-23'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-21_11-19-2'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_21-2-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_14-39-31'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_17-17-6'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_15-46-47'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_16-2-46'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_15-38-2'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-05_19-26-43'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-17_12-33-47'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_21-26-8'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_14-49-24'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_12-48-38'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_12-17-27'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-16_15-12-23'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-07_11-26-53'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-26_13-51-50'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-04_21-20-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_12-35-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_15-23-32'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_20-28-3'),
# ]


In [ ]:
completely_new_included_session_contexts = set(new_included_session_contexts) - set(old_included_session_contexts)
print_identifying_context_array_code(completely_new_included_session_contexts, array_name='completely_new_included_session_contexts')

# Testing programmatic .py to .ipynb conversion

In [ ]:
from pyphocorehelpers.notebook_helpers import convert_script_to_notebook

# Usage:
script_path = Path(r"K:\scratch\gen_scripts\run_kdiba_gor01_one_2006-6-12_15-55-31\run_kdiba_gor01_one_2006-6-12_15-55-31.py").resolve()
script_dir = script_path.parent.resolve()
notebook_path = script_path.with_suffix('.ipynb')
convert_script_to_notebook(script_path, notebook_path)
# convert_script_to_notebook(script_path, notebook_path, custom_delimiter=None)


# Old File Cleanup

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import OldFileArchiver


# deleted_file_list = OldFileArchiver.remove_backup_files_from_session_data_folders(good_session_concrete_folders=all_known_session_concrete_folders, is_dryrun=True)
always_delete_patterns = {
	'.pkltmp': '*.pkltmp',
	'.pkl.bak': '*.pkl.bak',
}
# always_delete_patterns = {} # disable non-conditional deletes while the pipeline is running in batch

# Define patterns to conditionally delete
conditional_delete_patterns = {
	# '.pkl': '*.pkl',
	'pipeline.pkl': '*loadedSessPickle*.pkl',
	'global.pkl': '*global_computation_results*.pkl',
}

# Define the cutoff date
# cutoff = datetime(2023, 9, 1)  # Example cutoff date
# cutoff = datetime(2024, 9, 1)
cutoff = datetime(2025, 2, 20)
# deleted_file_list = OldFileArchiver.remove_backup_files_from_session_data_folders(good_session_concrete_folders=all_known_session_concrete_folders, cutoff_date=cutoff, is_dryrun=True)
deleted_file_list = OldFileArchiver.remove_backup_files_from_session_data_folders(good_session_concrete_folders=all_known_session_concrete_folders,
																				  always_delete_patterns=always_delete_patterns,
																				  conditional_delete_patterns=conditional_delete_patterns,
																				   cutoff_date=cutoff, is_dryrun=False)
deleted_file_list

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import OldFileArchiver

## INPUTS: collected_outputs_directory, excluded_or_outdated_files_list
archive_folder = OldFileArchiver.archive_old_files(collected_outputs_directory=collected_outputs_directory, excluded_or_outdated_files_list=excluded_or_outdated_files_list, is_dry_run=True)

archive_folder